In [1]:
import numpy as np
from numpy.linalg import norm
from numpy import arctan, arcsin, arccos, cos, sin, dot, cross, matmul, arctan2

In [2]:
def Euler_Angles(angles):
    alfa, beta, gama = angles
    
    Rx = np.array([[1,         0,          0],
                   [0, cos(alfa), -sin(alfa)],
                   [0, sin(alfa),  cos(alfa)]])
    
    Ry = np.array([[cos(beta),  0, sin(beta)],
                   [        0,  1,         0],
                   [-sin(beta), 0, cos(beta)]])
    
    Rz = np.array([[cos(gama), -sin(gama), 0],
                   [sin(gama),  cos(gama), 0],
                   [0,                  0, 1]])
    
    A = np.matmul(Rz, matmul(Ry, Rx))
    return np.array(A)

In [3]:
def Matrix_AxisAngle(A):
    # Calc p
    M = A - np.identity(3)
    
    # Check if rows are not same to the factor
    row1 = np.array(M[0] / M[0][0])
    row2 = np.array(M[1] / M[1][0])
    row3 = np.array(M[2] / M[2][0])
    
    if not np.array_equal(row1, row2):
        p = cross(row1, row2)
    else:
        p = cross(row1, row3)
        
    p = p/norm(p)
    
    # Calc normal vector on p
    if p[0] != 0:
        u = [-(p[1] + p[2])/p[0], 1, 1]
    elif p[1] != 0:
        u = [1, -(p[0] + p[2])/p[1], 1]
    elif p[2] != 0:
        u = [1, 1, -(p[0] + p[1])/p[2]]
        
    u = u / norm(u)
    
    # Calc u projection
    up = matmul(A, u)
    
    # Calc angle
    fi = arccos( dot(u, up) / (norm(u)*norm(up)) )
    
    # Check positive orientation
    if dot(cross(u, up), p) < 0:
        p = -p
    
    return np.array(p), fi

In [4]:
def Rodrigez(p, fi):
    px = np.array([[    0, -p[2],  p[1]],
                   [ p[2],     0, -p[0]],
                   [-p[1],  p[0],     0]])
    p = np.matrix(p)
    C1 = matmul(p.T, p)
    C2 = cos(fi)*(np.identity(3) - C1)
    C3 = sin(fi)*px
                  
    A = C1 + C2 + C3
    return np.array(A)  

In [5]:
def Matrix_Angles(A):
    if A[2][0] < 1:
        if A[2][0] > -1:  # unique solution
            alfa = arctan2(A[2][1], A[2][2])
            beta = arcsin(-A[2][0])
            gama = arctan2(A[1][0], A[0][0])
            print('unique')
        else:            # not unique: case Ox3 = -Oz
            alfa = 0
            beta = np.pi / 2
            gama = arctan2(-A[0][1], A[1][1])
            print('not unique')
    else:                # not unique: case Ox3 = Oz
            alfa = 0
            beta = -np.pi / 2
            gama = arctan2(-A[0][1], A[1][1])
            print('not unique')
    
    return np.array([alfa, beta, gama])     

In [6]:
def AngleAxis_Quaternion(p, fi):
    w = cos(fi/2)
    
    p = p / norm(p)
    x, y, z = sin(fi/2)*p
    
    return np.array([x, y, z, w])

In [7]:
def Quaternion_AngleAxis(q):
    q = q / norm(q)
    
    if q[3]<0:
        q = -q
    
    fi = 2*arccos(q[3])
    
    if q[3]==1:
        p = [1, 0, 0]
    else:
        p = [q[0], q[1], q[2]]
        p = p / norm(p)
    
    return np.array(p), fi

### Testing 1

In [8]:
angles = [-arctan(1/4), -arcsin(8/9), arctan(4)]
angles

[-0.24497866312686414, -1.0949140771344799, 1.3258176636680326]

In [9]:
A = Euler_Angles(angles)
A

array([[ 0.11111111, -0.88888889, -0.44444444],
       [ 0.44444444,  0.44444444, -0.77777778],
       [ 0.88888889, -0.11111111,  0.44444444]])

In [10]:
if np.array_equal(matmul(A, A.T).round(5), np.identity(3)):
    print('Orthogonal')
else:
    print('ERROR: Not orthogonal matrix')
    print(matmul(A, A.T))

Orthogonal


In [11]:
print(f'Determinant equals:{np.linalg.det(A)}')

Determinant equals:0.9999999999999999


In [12]:
p, fi = Matrix_AxisAngle(A)
p, fi

(array([ 0.33333333, -0.66666667,  0.66666667]), 1.5707963267948966)

In [13]:
A_calculated = Rodrigez(p, fi)
A_calculated

array([[ 0.11111111, -0.88888889, -0.44444444],
       [ 0.44444444,  0.44444444, -0.77777778],
       [ 0.88888889, -0.11111111,  0.44444444]])

In [14]:
angles_calculated = Matrix_Angles(A_calculated)
angles_calculated

unique


array([-0.24497866, -1.09491408,  1.32581766])

In [15]:
q = AngleAxis_Quaternion(p, fi)
q

array([ 0.23570226, -0.47140452,  0.47140452,  0.70710678])

In [16]:
p, fi = Quaternion_AngleAxis(q)
p, fi

(array([ 0.33333333, -0.66666667,  0.66666667]), 1.5707963267948966)

### Testing 2

In [17]:
angles = [np.pi/3, np.pi/3, np.pi/3]
angles

[1.0471975511965976, 1.0471975511965976, 1.0471975511965976]

In [18]:
A = Euler_Angles(angles)
A

array([[ 0.25      , -0.0580127 ,  0.96650635],
       [ 0.4330127 ,  0.89951905, -0.0580127 ],
       [-0.8660254 ,  0.4330127 ,  0.25      ]])

In [19]:
if np.array_equal(matmul(A, A.T).round(5), np.identity(3)):
    print('Orthogonal')
else:
    print('ERROR: Not orthogonal matrix')
    print(matmul(A, A.T))

Orthogonal


In [20]:
print(f'Determinant equals:{np.linalg.det(A)}')

Determinant equals:1.0000000000000002


In [21]:
p, fi = Matrix_AxisAngle(A)
p, fi

(array([0.25056281, 0.93511313, 0.25056281]), 1.3696838321801164)

In [22]:
A_calculated = Rodrigez(p, fi)
A_calculated

array([[ 0.25      , -0.0580127 ,  0.96650635],
       [ 0.4330127 ,  0.89951905, -0.0580127 ],
       [-0.8660254 ,  0.4330127 ,  0.25      ]])

In [23]:
angles_calculated = Matrix_Angles(A_calculated)
angles_calculated

unique


array([1.04719755, 1.04719755, 1.04719755])

In [24]:
q = AngleAxis_Quaternion(p, fi)
q

array([0.15849365, 0.59150635, 0.15849365, 0.77451905])

In [25]:
p, fi = Quaternion_AngleAxis(q)
p, fi

(array([0.25056281, 0.93511313, 0.25056281]), 1.3696838321801161)

### Testing 3

In [26]:
angles = [np.pi/2, np.pi/3, np.pi/3]
angles

[1.5707963267948966, 1.0471975511965976, 1.0471975511965976]

In [27]:
A = Euler_Angles(angles)
A

array([[ 2.50000000e-01,  4.33012702e-01,  8.66025404e-01],
       [ 4.33012702e-01,  7.50000000e-01, -5.00000000e-01],
       [-8.66025404e-01,  5.00000000e-01,  3.06161700e-17]])

In [28]:
if np.array_equal(matmul(A, A.T).round(5), np.identity(3)):
    print('Orthogonal')
else:
    print('ERROR: Not orthogonal matrix')
    print(matmul(A, A.T))

Orthogonal


In [29]:
print(f'Determinant equals:{np.linalg.det(A)}')

Determinant equals:1.0


In [30]:
p, fi = Matrix_AxisAngle(A)
p, fi

(array([0.66666667, 0.66666667, 0.33333333]), 1.5676288169515635)

In [31]:
A_calculated = Rodrigez(p, fi)
A_calculated

array([[ 0.44620417,  0.109705  ,  0.88818165],
       [ 0.77636833,  0.44620417, -0.44514499],
       [-0.44514499,  0.88818165,  0.11392667]])

In [32]:
angles_calculated = Matrix_Angles(A_calculated)
angles_calculated

unique


array([1.44322338, 0.46133617, 1.04916305])

In [33]:
q = AngleAxis_Quaternion(p, fi)
q

array([0.47065734, 0.47065734, 0.23532867, 0.70822578])

In [34]:
p, fi = Quaternion_AngleAxis(q)
p, fi

(array([0.66666667, 0.66666667, 0.33333333]), 1.5676288169515635)